# Extract and display a synapse property matrix from a SONATA circuit

Copyright (c) 2025 Open Brain Institute

Authors: Christoph Pokorny

Last modified: 01.2025

## Summary
This analysis extracts and visualizes a matrix of a selected synapse property (conductance, delay, ...), grouped by a selected neuron property (layer, m-type, ...).
For details, see the [README](README.md).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from bluepysnap import Circuit
from connectome_manipulator.connectome_comparison import properties
from ipywidgets import widgets, interact

## Circuit selection

A SONATA circuit is assumed to be located under `./analysis_circuit/circuit_config.json`. For using an existing circuit from some other location, modify the `circuit_config = ...` path below accordingly.

In [2]:
# Path to existing circuit config
circuit_config = "analysis_circuit/circuit_config.json"

assert os.path.exists(circuit_config), f"ERROR: Circuit config '{os.path.split(circuit_config)[1]}' not found!"

Loading SONATA circuit. Selections of the edge population containing the synapses, as well as pre-/post-synaptic node sets defining groups of neurons are possible.

In [3]:
c = Circuit(circuit_config)
e_populations = c.edges.population_names
assert len(e_populations) > 0, "ERROR: No edge population found!"
node_sets = list(c.node_sets.content.keys())
e_popul_wdgt = widgets.Dropdown(options=e_populations, description="Edge population:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
pre_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Pre-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
post_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Post-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(e_popul_wdgt)
display(pre_nset_wdgt)
display(post_nset_wdgt)

Dropdown(description='Edge population:', layout=Layout(width='max-content'), options=('nodeA__nodeA__chemical'…

Dropdown(description='Pre-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'LayerA', 'L…

Dropdown(description='Post-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'LayerA', '…

## Grouping selection

The synaptic connectivity is grouped by one selected (categorical) neuron property, like layer, m-type, etc.

In [4]:
def get_props(nodes):
    is_cat = nodes.get([]).dtypes.apply(lambda _x: isinstance(_x, pd.CategoricalDtype))
    props = is_cat[is_cat].index.to_list()  # Select categorical properties
    for _p in ["region", "layer", "mtype", "etype", "synapse_class"]:
        if _p in nodes.property_names:
            props = props + [_p]  # Add certain properties, even if not categorical
    return np.unique(props)
node_props = np.intersect1d(get_props(c.edges[e_popul_wdgt.value].source), get_props(c.edges[e_popul_wdgt.value].target))
groupby_wdgt = widgets.Dropdown(options=node_props, value="layer" if "layer" in node_props else node_props[0], description="Group-by:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(groupby_wdgt)

Dropdown(description='Group-by:', index=1, layout=Layout(width='max-content'), options=(np.str_('etype'), np.s…

## Synapse properties extraction

Extract the synapse properties for all pairs of groups using functionality from [connectome-manipulator](https://github.com/openbraininstitute/connectome-manipulator).

In [5]:
props_dict = properties.compute(c, sel_src={"node_set": pre_nset_wdgt.value}, sel_dest={"node_set": post_nset_wdgt.value}, edges_popul_name=e_popul_wdgt.value, group_by=groupby_wdgt.value)
# TODO: fct=...

INFO: Extracting synapse properties (group_by=layer, sel_src={'node_set': None}, sel_dest={'node_set': None}, N=3x3 groups, per_conn=False)
INFO: Available synapse properties: 
['@source_node', '@target_node', 'afferent_center_x', 'afferent_center_y', 'afferent_center_z', 'afferent_section_id', 'afferent_section_pos', 'afferent_section_type', 'afferent_segment_id', 'afferent_segment_offset', 'afferent_surface_x', 'afferent_surface_y', 'afferent_surface_z', 'conductance', 'conductance_scale_factor', 'decay_time', 'delay', 'depression_time', 'efferent_center_x', 'efferent_center_y', 'efferent_center_z', 'efferent_section_id', 'efferent_section_pos', 'efferent_section_type', 'efferent_segment_id', 'efferent_segment_offset', 'efferent_surface_x', 'efferent_surface_y', 'efferent_surface_z', 'facilitation_time', 'n_rrp_vesicles', 'spine_length', 'syn_type_id', 'u_hill_coefficient', 'u_syn']


100% |###############################################################################################################################|


## Interactive visualization

Interactive visualization of the synapse property matrix. The user can select what property to display. Also, empty groups (i.e., groups w/o any neurons or connections) can be excluded using a checkbox.

In [6]:
# Interactive plot function
def plot_fct(res_sel, empty_sel):
    def filter_props_dict(props_dict, res_sel):
        out_dict = {}
        p = props_dict["conductance"]["data"]  # TODO: FIXME (better solution w/o using conductance hard-coded)
        pre_sel = np.any(p > 0.0, axis=1)
        post_sel = np.any(p > 0.0, axis=0)
        out_dict[res_sel] = props_dict[res_sel].copy()
        out_dict[res_sel]["data"] = out_dict[res_sel]["data"][pre_sel, :][:, post_sel]
        out_dict["common"] = props_dict["common"].copy()
        out_dict["common"]["src_group_values"] = [_v for _v, _s in zip(out_dict["common"]["src_group_values"], pre_sel) if _s]
        out_dict["common"]["tgt_group_values"] = [_v for _v, _s in zip(out_dict["common"]["tgt_group_values"], post_sel) if _s]
        return out_dict
    
    if empty_sel:
        plot_dict = props_dict
    else:
        plot_dict = filter_props_dict(props_dict, res_sel)

    if plot_dict[res_sel]["data"].size > 0:
        vmax = np.nanmax(plot_dict[res_sel]["data"])
        if vmax == 0.0:
            vmax = 1.0
        properties.plot(plot_dict[res_sel], plot_dict["common"], vmin=0.0, vmax=vmax, group_by=groupby_wdgt.value)
    else:
        plt.figure()
        plt.text(0, 0, "Nothing to show...")
        plt.axis("off")
        plt.axis("tight")
        plt.show()

In [7]:
def prop_incl(prop):
    """Exclude certain properties"""
    for _p in ["afferent_", "efferent_", "@", "_id"]:
        if _p in prop:
            return False
    return True
res_sel_opt = [(_v["unit"], _k) for _k, _v in sorted(props_dict.items()) if "unit" in _v and prop_incl(_k)]
res_sel_wdgt = widgets.Dropdown(options=res_sel_opt, description="Display:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
empty_sel_wdgt = widgets.Checkbox(value=True, description="Show empty groups", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
iplot = interact(plot_fct, res_sel=res_sel_wdgt, empty_sel=empty_sel_wdgt)

interactive(children=(Dropdown(description='Display:', layout=Layout(width='max-content'), options=(('Mean con…